In [1]:
# Load AIEEE Data

In [1]:
!nvidia-smi

Fri Mar 18 23:51:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   38C    P0    45W / 250W |   5988MiB / 16280MiB |      3%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"    
import tensorflow as tf

In [3]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


import sys  
sys.path.insert(0, 'Models/CBSEData/CBSEPreprocessing/')
from parse_cbse import *

In [5]:
cbse_df = CBSEData()

Load Models

In [6]:
#  other models
AIEEE_BERT = "Models/AIEEEGenderData/BERT/SavedModel/BERT"
CBSE_BERT = "Models/CBSEData/BERT/SavedModel/BERT"
ER_BERT = "Models/ERData/BERT/SavedModel/BERT"

In [7]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [8]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [9]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_BERT)

In [10]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_BERT)

In [11]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_BERT)

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


Predict Gender

In [13]:
from transformers import pipeline
from sklearn.metrics import classification_report


In [21]:
import math

def print_results(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
#     import pdb; pdb.set_trace()
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    print(classification_report(true, labels, digits = 4))
    return 

In [20]:
print_results(cbse_model, tokenizer, cbse_df)

ValueError: Found input variables with inconsistent numbers of samples: [152489, 152000]

In [ ]:
print_results(aieee_model, tokenizer, cbse_df)

In [ ]:
print_results(er_model, tokenizer, cbse_df)